In [14]:
# !pip install transformers -qqq

In [15]:
# from google.colab import drive
import math
import random
import time
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from tokenizers import Tokenizer
from PIL import Image
import numpy as np
from transformers import CLIPProcessor
import random
from torchvision import transforms

In [16]:
SEED = 2424

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [17]:
tokenizer_path ="/home/ivlabs/Documents/Kshitij/archive/Flickr_tokenizer.json"
tokenizer = Tokenizer.from_file(tokenizer_path)
print(tokenizer.get_vocab_size())
tokenizer.enable_padding(pad_id=4)

5197


In [18]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


class CaptioningDataset(Dataset):
  def __init__(self, split='train'):
    super().__init__()
    self.split=split
    
    data_path = "/home/ivlabs/Documents/Kshitij/archive/captions.txt"
    self.images_path = "/home/ivlabs/Documents/Kshitij/archive/Images/"

    with open(data_path) as f:
      lines = f.readlines()

    lines = lines[1:]
    random.shuffle(lines)
    
    images=[]
    captions=[]

    for some in lines:
      i,c = some.split(',',1)
      images.append(i)
      captions.append(c.rstrip('\n'))
      
    # images = images[1:]
    # captions = captions[1:]
    train_len = 30000

    test_len = (len(captions) - train_len)//2
    
    if self.split=='train':
      images = images[0:train_len]
      captions = captions[0:train_len]

    elif self.split=='test':
      images = images[train_len:train_len+test_len]
      captions = captions[train_len:train_len+test_len]

    elif self.split=='validation':
      images = images[train_len+test_len:train_len+(2*test_len)]
      captions = captions[train_len+test_len:train_len+(2*test_len)]

    self.images = images
    self.captions = captions

  def __len__(self):
    return len(self.images)

  # def __getitem__(self, index):
  #   # print('here')
  #   want_caption = self.captions[index]
  #   want_image = self.images_path + self.images[index]
  #   want_image = Image.open(want_image)
  #   want_image = np.array(want_image.resize((224,224))).reshape(-1,224,224)
  #   if want_image.shape[0]==1:
  #     want_image = np.concatenate((want_image,want_image,want_image),axis=0)
  #   # want_image = want_image.tolist()
  #   # print('here now')
  #   return want_image.tolist(), want_caption 

  def __getitem__(self, index):
    # print('here')
    want_caption = self.captions[index]
    want_image = self.images_path + self.images[index]
    want_img_location = want_image
    want_image = Image.open(want_image)
    # want_image = np.array(want_image.resize((224,224))).reshape(224,224,-1)
    # want_image = np.array(want_image.resize((224,224))).reshape(-1,224,224)
    want_image = preprocess(want_image)
    if want_image.shape[0]==1:
      want_image = np.concatenate((want_image,want_image,want_image),axis=0)
    # want_image = want_image.tolist()
    # print('here now')
    return want_image, want_caption, want_img_location

class MyCollate:
  def __init__(self):
    self.tokenizer = tokenizer

  def __call__(self,batch):
    images=[]
    captions=[]
    image_locations= []
    for i in batch:
      # print(i)
      images.append(i[0])
      captions.append(i[1])
      image_locations.append(i[2])
    
    # print(images)
    # print(captions)
    
    captions = self.tokenizer.encode_batch(captions)

    want_captions = []
    attn = []

    for i in captions:
      # print(i.ids)
      want_captions.append(i.ids)
      attn.append(i.attention_mask)
    # print(want_captions)
    want_captions = torch.Tensor(want_captions).int()
    attn = torch.Tensor(attn)
    # images = torch.Tensor(images)
    images = torch.stack(images)
    return images, want_captions, attn.T, image_locations

In [19]:

dataset = CaptioningDataset(split='train')
trainloader = DataLoader(dataset, batch_size=64, shuffle=True, collate_fn=MyCollate())
DEVICE = 'cuda'
for i in trainloader:
  img = i[0]
  # print(i[0].pixel_values.shape)
  text = i[1]
  # print(text.shape)
  # print(text)
  break
print(img.shape)
print(text.shape)

torch.Size([64, 3, 224, 224])
torch.Size([64, 36])


In [20]:
import torchvision
img_encoder_model = torchvision.models.resnet101(pretrained=True)
img_encoder_model = torch.nn.Sequential(*(list(img_encoder_model.children())[:-2]))

class Encoder(nn.Module):
  def __init__(self, img_dim, num_proj_layers):
    super().__init__()
    self.resnet_dim = 2048
    self.img_dim = img_dim
    self.img_enc = img_encoder_model

    self.num_proj_layers = num_proj_layers

    layers = []
    for i in range(num_proj_layers):
      if i==0:
        layers.append(nn.Sequential(nn.Linear(self.resnet_dim, self.img_dim),
                                    nn.ReLU()
                                    ))
      else:
        layers.append(nn.Sequential(nn.Linear(self.img_dim, self.img_dim),
                                    nn.ReLU()
                                    ))
    self.layers = nn.ModuleList(layers)  
  
  def forward(self, img):
    b = img.shape[0]
    img = self.img_enc(img)
    img = img.view(b,self.resnet_dim, -1).permute(0,2,1)
    # print(img.shape)
    img = img.squeeze()
    for i in range(self.num_proj_layers):
      img = self.layers[i](img)
    return img.squeeze()

# img_enc = Encoder(512,1)
# i = {"pixel_values":torch.randn(64,3,224,224)}
# out = img_enc(i)
# print(out.shape)

In [21]:
class MultiHead_Attn_Layer(nn.Module):
    def __init__(self, hidden_dim, n_heads, dropout):
        super(MultiHead_Attn_Layer, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_heads = n_heads
        self.head_dim = hidden_dim // n_heads
        self.fc_Q = nn.Linear(hidden_dim, hidden_dim)
        self.fc_K = nn.Linear(hidden_dim, hidden_dim)
        self.fc_V = nn.Linear(hidden_dim, hidden_dim)
        self.fc_O = nn.Linear(hidden_dim, hidden_dim)
        self.scale = math.sqrt(self.head_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value, mask=None):                                    # [query] = [batch_size, query_len, hidden_dim] [key] = [batch_size, key_len, hidden_dim] [value] = [batch_Size, value_len, hidden_dim]
        batch_size = query.shape[0]
        # print(key.shape)
        # print(query.shape)
        # print(value.shape)                                           
        Q = self.fc_Q(query)                                                            # [Q] = [batch_size, query_len, hidden_dim]   
        K = self.fc_K(key)                                                              # [K] = [batch_size, key_len, hidden_dim]
        V = self.fc_V(value)                                                            # [V] = [batch_size, value_len, hidden_dim]
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)     # [Q] = [batch_size, num_heads, query_len, head_dim]
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)     # [K] = [batch_size, num_heads, key_len, head_dim]
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)     # [V] = [batch_size, num_heads, value_len, head_dim]
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        if mask is not None:                                                            # [energy] = [batch_size, num_heads, query_len, key_len]  
            energy = energy.masked_fill(mask == False, -1e10)                               
        attention = torch.softmax(energy, dim = -1)                                     # [attention] = [batch_size, num_heads, query_len, key_len]
        x = torch.matmul(self.dropout(attention), V)                                    # [x] = [batch_size, num_heads, query_len, head_dim]
        x = x.permute(0, 2, 1, 3).contiguous()                                          # [x] = [batch_size, query_len, num_heads, head_dim]
        # Can avoid contiguous() if we use .reshape instead of .view in the next line
        out = self.fc_O(x.view(batch_size, -1, self.hidden_dim))                        # [out] = [batch_size, query_len, hidden_dim]   
        return out, attention

class Postn_Feed_Fwrd(nn.Module):
    def __init__(self, hidden_dim, pff_dim, dropout):
        super(Postn_Feed_Fwrd, self).__init__()
        self.fc1 = nn.Linear(hidden_dim, pff_dim)
        self.fc2 = nn.Linear(pff_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input):                                                   # input = [batch_size, seq_len, hidden_dim]
        out = torch.relu(self.fc1(input))                                       # out = [batch_size, seq_len, pff_dim]
        out = self.fc2(self.dropout(out))                                       # out = [batch_size, seq_len, hidden_dim] 
        return out

class Decoder_Layer(nn.Module):
    def __init__(self, hidden_dim, n_heads, pff_dim, dropout):
        super(Decoder_Layer, self).__init__()
        self.self_attn = MultiHead_Attn_Layer(hidden_dim, n_heads, dropout)
        self.cross_attn = MultiHead_Attn_Layer(hidden_dim, n_heads, dropout)
        self.pff = Postn_Feed_Fwrd(hidden_dim, pff_dim, dropout)
        self.attn_norm1 = nn.LayerNorm(hidden_dim)
        self.attn_norm2 = nn.LayerNorm(hidden_dim)
        self.pff_norm = nn.LayerNorm(hidden_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, trg, trg_mask, enc_out, src_mask):                            # trg = [batch_size, trg_len, hidden_dim] trg_mask = [batch_size, trg_len] enc_out = [batch_size, src_len, hidden_dim] src_mask = [batch_size, src_len]
        sattn_out, _ = self.self_attn(trg, trg, trg, trg_mask)                      # satten_out = [batch_size, trg_len, hidden_dim]
        inter_out1 = self.attn_norm1(self.dropout(sattn_out) + trg)                 # inter_out1 = [batch_size, trg_len, hidden_dim]  
        cattn_out, attn = self.cross_attn(inter_out1, enc_out, enc_out, src_mask)   # cattn_out = [batch_size, trg_len, hidden_dim] attn = [batch_size, num_heads, query_len, key_len]
        inter_out2 = self.attn_norm2(self.dropout(cattn_out) + inter_out1)          # inter_out2 = [batch_size, trg_len, hidden_dim]
        pff_out = self.pff(inter_out2)                                              # pff_out = [batch_size, trg_len, hidden_dim]
        out = self.pff_norm(self.dropout(pff_out) + inter_out2)                     # out = [batch_size, trg_len, hidden_dim]
        return out, attn


class Decoder(nn.Module):
    def __init__(self, tok_vocab_size, pos_vocab_size, hidden_dim, dec_heads, dec_pff_dim, num_layers, dec_dropout):
        super(Decoder, self).__init__()
        self.tok_embedding = nn.Embedding(tok_vocab_size, hidden_dim)
        self.pos_embedding = nn.Embedding(pos_vocab_size, hidden_dim) #pos_vocab_size = Max possible sequence length
        self.dec_layers = nn.ModuleList([Decoder_Layer(hidden_dim, dec_heads, dec_pff_dim, dec_dropout) for i in range(num_layers)])
        self.fc = nn.Linear(hidden_dim, tok_vocab_size)
        self.scale = math.sqrt(hidden_dim)
        self.dropout = nn.Dropout(dec_dropout)

    def forward(self, trg, trg_mask, enc_out, src_mask):                                    # trg = [batch_size, trg_len] trg_mask = [batch_size, trg_len] enc_out = [] src_mask = []
        batch_size = trg.shape[0] 
        trg_len = trg.shape[1]
        tok_embed = self.tok_embedding(trg)                                                 # tok_embed = [batch_size, trg_len, hidden_dim]
        pos_tensor = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(device) # pos_tensor = [batch_size, trg_len]
        pos_embed = self.pos_embedding(pos_tensor)                                          # pos_embed = [batch_size, trg_len, hidden_dim]
        dec_embed = self.dropout(tok_embed * self.scale + pos_embed)                        # dec_embed = [batch_size, trg_len, hidden_dim]
        dec_state = dec_embed
        for dec_layer in self.dec_layers:
            dec_state, attention = dec_layer(dec_state, trg_mask, enc_out, src_mask)        # dec_state = [batch_size, trg_len, hidden_dim] attention = [batch_size, num_heads, query_len, key_len]
        out = self.fc(dec_state)                                                            # out = [batch_size, tok_vocab_size]
        return out, attention

In [22]:
class Captioner(nn.Module):
  def __init__(self, 
               img_dim,         #image encoder
               num_proj_layers, #image encoder
               tok_vocab_size,  #output vocab size
               pos_vocab_size,  #max possible length of sentence
               hidden_dim,      
               dec_heads, 
               dec_pff_dim, 
               num_layers, 
               dec_dropout):
    
    super().__init__()
    self.trg_padding_idx = 4
    self.image_encoder = Encoder(img_dim=img_dim, num_proj_layers=num_proj_layers)
    self.language_model = Decoder(tok_vocab_size, pos_vocab_size, hidden_dim, dec_heads, dec_pff_dim, num_layers, dec_dropout)

  def make_trg_mask(self, trg):                                                       # trg = [batch_size, trg_len]                  
    trg_len = trg.shape[1] 
    pad_mask = (trg != self.trg_padding_idx).unsqueeze(1).unsqueeze(2).to(device)   # pad_mask = [batch_size, 1, 1, trg_len]
    sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = device)).bool()   # sub_mask = [trg_len, trg_len]
    trg_mask = pad_mask & sub_mask                                                  # trg_mask = [batch_size, 1, trg_len, trg_len]
    return trg_mask

  def make_src_mask(self, src):                                                       # src = [batch_size, src_len]
        # src_mask = (src != self.src_padding_idx).unsqueeze(1).unsqueeze(2).to(device)   # src_mask = [batch_size, 1, 1, src_len]
        src_mask = torch.ones(src.shape[0],1,1,src.shape[1]).bool().to(device)
        return src_mask

  def forward(self, image, caption, train=True):
    img = self.image_encoder(image)
    if train:
      trg_mask = self.make_trg_mask(caption)
    else:
      # print(caption.shape)
      trg_len = caption.shape[1]
      pad_mask = (caption != self.trg_padding_idx).unsqueeze(1).unsqueeze(2).to(device)   # pad_mask = [batch_size, 1, 1, trg_len]
      sub_mask = torch.ones(trg_len, trg_len).bool().to(device)   # sub_mask = [trg_len, trg_len]
      trg_mask = pad_mask & sub_mask               
    src_mask = self.make_src_mask(img)
    output, attention = self.language_model(caption, trg_mask, img, src_mask)
    return output, attention

In [23]:
def train(model, criterion, optimizer, iterator, clip=1, device='cuda'):
  epoch_loss=0
  for data in iterator:
    img = data[0].to(device)
    text = data[1].to(device)
    # print(img.shape)
    # print(text.shape)
    # print(text.shape)
    optimizer.zero_grad()
    model_input_text = text[:,:-1]
    model_output_text = text[:,1:]
    output, _ = model(img, model_input_text)
    output = output.view(-1, output.shape[-1])
    # print(output.shape)
    model_output_text = model_output_text.contiguous().view(-1)
    batch_loss = criterion(output, model_output_text.to(device).long())
    batch_loss.backward()
    # torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    epoch_loss += batch_loss.item()

  return epoch_loss/len(iterator)

In [24]:
def Epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return (elapsed_mins, elapsed_secs)

In [26]:
LR = 1e-3
model = Captioner(img_dim=256,         #image encoder
               num_proj_layers=1, #image encoder
               tok_vocab_size=5197,  #output vocab size
               pos_vocab_size=150,  #max possible length of sentence
               hidden_dim=256,      
               dec_heads=4, 
               dec_pff_dim=128, 
               num_layers=6, 
               dec_dropout=0.1).to('cuda')
criterion = nn.CrossEntropyLoss(ignore_index = 4)
optimizer = optim.Adam(model.parameters(), LR)
dataset = CaptioningDataset(split='train')
trainloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=MyCollate())
device = 'cuda'

In [27]:
import os
MODEL_TYPE = "Transformer"
OUTPUT_PATH = f"/home/ivlabs/Documents/Kshitij/thanmay/models/{MODEL_TYPE}"
MODEL_STORE_PATH = os.path.join(OUTPUT_PATH,f"{MODEL_TYPE}_checkpoint_epoch.pth")
EPOCH_SAVE = 4 # Save the model every EPOCH_SAVE epochs
outfile = open(os.path.join(OUTPUT_PATH, f"{MODEL_TYPE}_train_losses.txt"), "a")
# outfile.write("Training Loss\tTraining PPL\n")

# model.load_state_dict(torch.load(MODEL_STORE_PATH.replace("epoch",str(40))))

train_losses = []
valid_losses = []
min_losses = 100
prev_epoch = 1
# min_losses = [float('inf'), float('inf')]
NUM_EPOCHS = 40
start_time = time.time()
for epoch in range(1,41):
    train_loss = train(model, criterion=criterion, optimizer=optimizer, iterator=trainloader)
    train_losses.append(train_loss)
    if epoch % EPOCH_SAVE == 0:
        torch.save(model.state_dict(), MODEL_STORE_PATH.replace("epoch",str(epoch)))
    elapsed_time = Epoch_time(start_time, time.time())
    print(f"Time taken for epochs {prev_epoch} to {epoch}: {elapsed_time[0]}m {elapsed_time[1]}s")
    start_time = time.time()
    prev_epoch = epoch + 1
    print(f"Training Loss: {train_loss:.4f} ")
    print(f"Training PPL: {math.exp(train_loss):.4f} ")
    outfile.write(f"{train_loss:.4f}\t{                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             math.exp(train_loss):.4f}\n")
outfile.close()
# print(f"Model with Train Loss {min_losses[1]:.4f}, Validation Loss: {min_losses[0]:.4f} was saved.")

Time taken for epochs 1 to 1: 8m 5s
Training Loss: 4.5211 
Training PPL: 91.9398 
Time taken for epochs 2 to 2: 8m 9s
Training Loss: 3.8590 
Training PPL: 47.4194 
Time taken for epochs 3 to 3: 8m 1s
Training Loss: 3.6771 
Training PPL: 39.5333 
Time taken for epochs 4 to 4: 8m 1s
Training Loss: 3.5634 
Training PPL: 35.2841 
Time taken for epochs 5 to 5: 7m 59s
Training Loss: 3.4723 
Training PPL: 32.2100 
Time taken for epochs 6 to 6: 7m 58s
Training Loss: 3.3980 
Training PPL: 29.9056 
Time taken for epochs 7 to 7: 8m 0s
Training Loss: 3.3344 
Training PPL: 28.0608 
Time taken for epochs 8 to 8: 7m 59s
Training Loss: 3.2765 
Training PPL: 26.4842 
Time taken for epochs 9 to 9: 8m 4s
Training Loss: 3.2285 
Training PPL: 25.2427 
Time taken for epochs 10 to 10: 7m 57s
Training Loss: 3.1806 
Training PPL: 24.0609 
Time taken for epochs 11 to 11: 7m 59s
Training Loss: 3.1374 
Training PPL: 23.0450 
Time taken for epochs 12 to 12: 7m 58s
Training Loss: 3.0981 
Training PPL: 22.1556 
Time

In [ ]:
def testing(model, iterator, tokenizer):
    predictions = []
    locations = []
    captions = []
    model.eval()
    with torch.no_grad():
        for data in enumerate(iterator):
            batch_locations = data[1][-1]
            img = data[1][0]
            text = data[1][1].to(device)
            batch_size = text.shape[0]
            img = img.to(device)
            output, _ = model(img, text, train=True)
            output = torch.softmax(output, dim=-1)
            output = torch.argmax(output, dim=-1)
            predictions.extend(tokenizer.decode_batch(output.tolist()))
            captions.extend(tokenizer.decode_batch(text.tolist()))
            locations.extend(batch_locations)
    
    return predictions, locations, captions

In [ ]:

test_dataset = CaptioningDataset(split='test')
testloader = DataLoader(test_dataset, batch_size=64, shuffle=True, collate_fn=MyCollate())

In [ ]:
# !pip install evaluate -qqq
# !pip install rouge_score -qqq

import evaluate

meteor = evaluate.load('meteor')
rouge = evaluate.load('rouge')
bleu = evaluate.load('bleu')

2023-03-27 19:43:56.827715: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-27 19:43:57.916411: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/noetic/lib::/home/ivlabs/.mujoco/mjpro150/bin
2023-03-27 19:43:57.916485: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/noetic/lib::/home/ivlabs/.mujoco/mjpro150/bin
2023-03-27 19:43:57.

In [ ]:
import os
MODEL_TYPE = "Transformer"
OUTPUT_PATH = f"/home/ivlabs/Documents/Kshitij/thanmay/models/{MODEL_TYPE}"
MODEL_STORE_PATH = os.path.join(OUTPUT_PATH,f"{MODEL_TYPE}_checkpoint_epoch.pth")
EPOCH_SAVE = 4 # Save the model every EPOCH_SAVE epochs
outfile = open(os.path.join(OUTPUT_PATH, f"{MODEL_TYPE}_scores.txt"), "w")
outfile.write("EPOCH\tBLEU\tMETEOR\tROUGE1\nROUGE2\tROUGE_L\tROUGE_Lsum\n")

NUM_EPOCHS = 40
for epoch in range(EPOCH_SAVE, NUM_EPOCHS + 1, EPOCH_SAVE):
    model.load_state_dict(torch.load(MODEL_STORE_PATH.replace("epoch",str(epoch))))
    predictions, locations, captions = testing(model,testloader,tokenizer)
    bleu_results = bleu.compute(predictions=predictions, references=captions)
    meteor_results = meteor.compute(predictions=predictions, references=captions)
    rouge_results = rouge.compute(predictions=predictions, references=captions)
    outfile.write(f"{epoch}\t{bleu_results['bleu']}\t{meteor_results['meteor']}\t{rouge_results['rouge1']}\t{rouge_results['rouge2']}\t{rouge_results['rougeL']}\t{rouge_results['rougeLsum']}\n")    
outfile.close()
